In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import torch
import gc
import os
import sys
%matplotlib notebook
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('.')) + '/src'
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from criteria import KIDMindLoss
from helpers import Trainer
from helpers import Logger
from helpers import Trial
from dataloader import KID_Data
from modules import Mind_of_KID
import utils

In [ ]:
# Load the data
main_dir = '/home/hazmat/GitHub/KID/'
train_dir = main_dir + 'datasets/KID_memory'
# test_dir = main_dir + 'low_AD/'
modl_dir = main_dir + 'models/'
rslt_dir = main_dir + 'results/'

train_ds = KID_Data(folder=train_dir, crop=55, scale=1, stride=1)
# test_ds = MIBIData(folder=test_dir, crop=32, scale=10, stride=16)

In [ ]:
kidmind = Mind_of_KID()

# kidmind = Mind_of_KID.load_model(main_dir + 'models/kinematics/', 'model_2999')
if torch.cuda.is_available():
    kidmind.cuda()
    pass

In [ ]:
kidmind_logger = Logger(['loss'])
kidmind_trainer = Trainer()

In [ ]:
# LadderNet training parameters
kidmind_train_args = dict()
kidmind_train_args['lr'] = 0.001
kidmind_train_args['batch_size'] = 1000
kidmind_train_args['epochs'] = 1000
kidmind_train_args['report'] = 5
kidmind_train_args['crop'] = 55
kidmind_train_args['clip'] = None
kidmind_train_args['decay'] = 0
kidmind_train_args['restart'] = -1
kidmind_train_args['epoch_frac'] = 1
# laddernet_train_args['decay'] = 1e-5

# LadderNet loss parameters
kidmind_loss_args = {
    'alpha': 1, # sensory loss
    'beta': 1, # inverse loss
    'gamma': 1, # forward loss
    'delta': 1 # perceptual loss
}

train_ds.set_crop(kidmind_train_args['crop'])

In [ ]:
kidmind_train_args['continue'] = False
kidmind.set_noise_level(0)
kidmind_criterion = KIDMindLoss(**kidmind_loss_args)
kidmind_trainer.train(kidmind, train_ds, kidmind_criterion, kidmind_logger, '/home/hazmat/GitHub/KID/models/kinematics/', **kidmind_train_args)

In [ ]:
import numpy as np
import time
kidmind.set_noise_level(0)
kidmind.eval()

idx1 = 20
idx2 = 48

batch1 = train_ds.get_samples([idx1])
batch2 = train_ds.get_samples([idx2])

batch1 = kidmind.forward(**batch1)
batch2 = kidmind.forward(**batch2)

img1 = batch1['~s_i'][0,:,:,:].transpose(0,1).transpose(1,2).detach()
img2 = batch2['~s_i'][0,:,:,:].transpose(0,1).transpose(1,2).detach()

p = batch2['p_i']
print(p)

fig = plt.figure(figsize=[10,5])
ax1 = plt.subplot(1,2,1)
ax2 = plt.subplot(1,2,2)

ax1.imshow(img1.cpu())
ax2.imshow(img2.cpu())